In [2]:
import cv2
import pytesseract
import numpy as np
import re
import json

In [11]:
def preprocess_image(image_path):
    """Load and preprocess the image for OCR."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    img = cv2.GaussianBlur(img, (3, 3), 0)  # Reduce noise
    img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]  # Apply thresholding
    return img

def extract_text(image):
    """Extract text from an image using Tesseract OCR."""
    custom_config = r'--oem 3 --psm 6'  # OCR Engine Mode (OEM) & Page Segmentation Mode (PSM)
    text = pytesseract.image_to_string(image, config=custom_config)
    return text

def extract_key_information(text):
    """Extract key aviation details using regex."""
    data = {}

    # Extract ICAO/IATA Airport Code
    match = re.search(r'\b([A-Z]{4})\b', text)
    data["Airport Code"] = match.group(1) if match else "Unknown"

    # Extract Approach Type (e.g., ILS, RNAV, VOR)
    match = re.search(r'(ILS|RNAV|VOR|NDB)\s+(?:RWY\s*\d+)', text, re.IGNORECASE)
    data["Approach Type"] = match.group(1) if match else "Unknown"

    # Extract Runway Number
    match = re.search(r'RWY\s*(\d{2}[LRC]?)', text, re.IGNORECASE)
    data["Runway"] = match.group(1) if match else "Unknown"

    # Extract ILS Frequency
    match = re.search(r'ILS\s+([\d.]+)\s+MHz', text, re.IGNORECASE)
    data["ILS Frequency"] = match.group(1) if match else "Unknown"

    # Extract Minimum Descent Altitude (MDA) or Decision Altitude (DA)
    match = re.search(r'(DA|MDA)\s*(\d+)', text, re.IGNORECASE)
    data["Minimum Descent Altitude"] = match.group(2) if match else "Unknown"

    return data

def main(image_path):
    """Complete OCR Pipeline"""
    print("[INFO] Preprocessing Image...")
    processed_image = preprocess_image(image_path)

    print("[INFO] Extracting Text...")
    extracted_text = extract_text(processed_image)

    print("[INFO] Extracting Key Information...")
    key_data = extract_key_information(extracted_text)

    print("[INFO] Extraction Complete!")
    print(json.dumps(key_data, indent=4))

    return key_data

# Run the pipeline with an approach chart image
image_path = "/Users/seaqueue/Documents/coop/my startup/pilot_asistant/pamodel/pudong.png" 
main(image_path)

[INFO] Preprocessing Image...
[INFO] Extracting Text...
[INFO] Extracting Key Information...
[INFO] Extraction Complete!
{
    "Airport Code": "Unknown",
    "Approach Type": "Unknown",
    "Runway": "Unknown",
    "ILS Frequency": "Unknown",
    "Minimum Descent Altitude": "Unknown"
}


{'Airport Code': 'Unknown',
 'Approach Type': 'Unknown',
 'Runway': 'Unknown',
 'ILS Frequency': 'Unknown',
 'Minimum Descent Altitude': 'Unknown'}